INSTALLING THE PACKAGES

In [ ]:
!pip install opencv-python

LOADING THE LIBRARIES


In [ ]:
import cv2
import numpy as np
from keras.utils import np_utils
import os


UNZIPPING THE DATASET

In [ ]:
!unzip /content/dataset-20210221T080358Z-001.zip

Archive:  /content/dataset-20210221T080358Z-001.zip
replace dataset/without_mask/262.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

LOADING THE DATASET

In [ ]:
data_path= '/content/dataset'
os.path.exists(data_path)

True

EXTRACTING FEATURES AND TARGET

In [ ]:
categories= os.listdir(data_path)
labels =[i for i in range(len(categories))]
label_dict =dict(zip(categories,labels)) 
print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [ ]:
img_size=100
data=[]
target=[]

In [ ]:
for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
        resized=cv2.resize(gray,(img_size,img_size))
        data.append(resized)
        target.append(label_dict[category]) 

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [ ]:
np.save('data',data)
np.save('target',new_target)

In [ ]:
data=np.load('data.npy')
target=np.load('target.npy')

ARCHITECTURE OF THE MODEL

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [ ]:
model=Sequential()

In [ ]:

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

COMPILING THE MODEL

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

TRAINING, TESTING AND FITTING THE MODEL

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
28/28 [==============================] - 31s 1s/step - loss: 3.0461 - accuracy: 0.5060 - val_loss: 0.6721 - val_accuracy: 0.4955
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/20
28/28 [==============================] - 41s 1s/step - loss: 0.6460 - accuracy: 0.6103 - val_loss: 0.5590 - val_accuracy: 0.7364
INFO:tensorflow:Assets written to: model-002.model/assets
Epoch 3/20
28/28 [==============================] - 30s 1s/step - loss: 0.4772 - accuracy: 0.7722 - val_loss: 0.4344 - val_accuracy: 0.7955
INFO:tensorflow:Assets written to: model-003.model/assets
Epoch 4/20
28/28 [==============================] - 31s 1s/step - loss: 0.3174 - accuracy: 0.8710 - val_loss: 0.4101 - val_accuracy: 0.7955
INFO:tensorflow:Assets written to: model-004.model/assets
Epoch 5/20
28/28 [==============================] - 30s 1s/step - loss: 0.2510 - accuracy: 0.8960 - val_loss: 0.2717 - val_accuracy: 0.9091
INFO:tensorflow:Assets written to: model-005.model/assets
Epoch 6/20

EVALUATING THE MODEL

In [ ]:
print(model.evaluate(test_data,test_target))

9/9 [==============================] - 2s 189ms/step - loss: 0.4030 - accuracy: 0.8986
[0.40299472212791443, 0.8985507488250732]


In [ ]:
from keras.models import load_model
import cv2
import numpy as np

In [ ]:
model = load_model('model-015.model')
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
source=cv2.VideoCapture(0)

labels_dict={0:'  MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}